## Retriever

In [1]:
from langchain_community.document_loaders import DirectoryLoader

In [2]:
from langchain_community.vectorstores import Chroma
import chromadb
from langchain_community.embeddings import HuggingFaceEmbeddings

In [3]:
embeddings = HuggingFaceEmbeddings()

/home/vi-b-04/CSEChatbot/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/home/vi-b-04/CSEChatbot/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [6]:
client = chromadb.HttpClient(host='127.0.0.1', port=1234)
db = Chroma(collection_name='test', embedding_function=embeddings, client=client)

In [7]:
retriever = db.as_retriever()

In [8]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

### Runnable Pass Through

In [9]:
from langchain_core.runnables import RunnablePassthrough

## Prompt Template

In [10]:
from langchain_core.prompts import ChatPromptTemplate

In [11]:
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            You are a helpful AI bot. 
            Answer the questions based on the context provided.

            {context}
            """
        ),
        ("human", "{question}"),
    ]
)

## LLM

In [12]:
from langchain_community.chat_models import ChatOllama

In [13]:
chat_model = ChatOllama(model='llama3')

## Output Parser

In [14]:
from langchain_core.output_parsers import StrOutputParser

In [15]:
output = StrOutputParser()

## Chain

In [16]:
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | chat_model
    | output
)

In [17]:
chain.invoke("When is the holiday for EID?")

'According to the notification, there is no mention of Eid-related holidays. However, it does mention that the Institute will observe a holiday on April 11, 2024, which might be due to Ramzan as per the MAHE circular dated November 24, 2023.'

In [18]:
chain.invoke("What is the assessment plan for this academic year?")

'According to Table 1: Schedule of In-Semester Assessment, the assessment plan for this academic year includes:\n\n1. MISAC 1 (Surprise Assignment): February 05-10, 2024 - 5 marks\n2. MISAC2 (Quiz): February 26-March 02, 2024 - 5 marks\n3. MISAC 3 (Mid-Term Examination): March 18-23, 2024 - 30 marks\n4. FISAC (Flexible In-Semester Assessment Component): This component includes three types of assessments: Quiz/MCQs, Surprise Assignment, and Take Home Assignment.\n\nNote that the exact dates for each assessment may be subject to change, but these are the planned dates according to the syllabus provided.'

In [19]:
for chunk in chain.stream('When is the holiday for Ramzan?'):
    print(chunk, end='')

According to the notification, the holiday for Ramzan will be on April 10, 2024.